In [1]:
using BoundaryValueDiffEq, ModelingToolkit, Plots, Printf
using ModelingToolkit: t_nounits as t, D_nounits as D

In [2]:
@mtkmodel TEARING begin
    @constants begin
        S = 100
        k = 0.5
    end
    @parameters begin
        γ = 0.05753900494
    end
    @variables begin
        # initialize with only BCs we know at t=0
        F(t)
        ψ(t) = 1
        φ(t) = 0
    end
    @equations begin
        F ~ tanh(t)
        S * γ * (ψ - F * φ) ~ (D(D(ψ)) - k^2 * ψ)
        γ^2 * (D(D(φ)) - k^2 * φ) ~ -F * D(D(ψ)) + F * k^2 * ψ + D(D(F)) * ψ
    end
end
@mtkcompile sys = TEARING()

Model sys:
Equations (4):
  4 standard: see equations(sys)
Unknowns (4): see unknowns(sys)
  φ(t) [defaults to 0]
  ψ(t) [defaults to 1]
  ψˍt(t)
  φˍt(t)
Parameters (3): see parameters(sys)
  γ [defaults to 0.057539]
  k [defaults to 0.5]
  S [defaults to 100]
Observed (3): see observed(sys)

In [3]:
L = 20
tspan = (0, L)
#=
const k = 0.5
@inline f(t) = tanh(t)
@inline ddf(t) = -2 * tanh(t) * sech(t)^2

ψouter(t) = exp(-k * abs(t)) * (1 + f(abs(t)) / k)
ϕouter(t) = ψouter(t) / f(t)
ψouter_prime(t) = -k * sign(t) * exp(-k * abs(t)) * (1 + f(abs(t)) / k) + exp(-k * abs(t)) * (ddf(abs(t)) / k)
ϕouter_prime(t) = ψouter_prime(t) / f(t) - ψouter(t) * ddf(t) / (f(t)^2)
=#

(0, 20)

In [4]:
mtkprob = BVProblem(sys, [D(sys.ψ) => 0, D(sys.φ) => 36.34514085047722], tspan, jac=true)

BVProblem with uType Vector{Float64} and tType Int64. In-place: true
Initialization status: FULLY_DETERMINED
Non-trivial mass matrix: false
timespan: (0, 20)
u0: 4-element Vector{Float64}:
  0.0
  1.0
  0.0
 36.34514085047722

In [ ]:
sol = solve(mtkprob, MIRK6(), dt=0.01, saveat=0.05)

In [ ]:
plot(sol)

In [ ]:
println("γ = ", sol.mtkprob.p[1])